In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic1")
sc   = SparkContext(conf=conf)

24/12/10 14:07:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 14:07:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc.stop()

In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic1")
sc   = SparkContext(conf=conf)

In [5]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [6]:
#관객수가 500만 이상인 영화를 가져오기 
1. 조인 > 필터 > 영화정보
2. 필터 > 조인 > 영화정보

SyntaxError: invalid syntax (4200590484.py, line 2)

In [7]:
#1. 조인먼저
movies_att = movies_rdd.join(attendances_rdd)
movies_att.filter(
    lambda x: x[1][1][0] > 5000000
).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [8]:
#2. 필터먼저
filtered_att = attendances_rdd.filter(lambda x: x[1][0]>5000000)
movies_rdd.join(filtered_att).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_sql_basic").getOrCreate()

In [10]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [11]:
movie_schema = ['id', 'name', 'company', 'year', 'month', 'day']

# 1. DataFrame 만들기

In [12]:
df = spark.createDataFrame(data=movies, schema= movie_schema)

In [13]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [14]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [15]:
df.select('name').show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [16]:
df.filter(df.year>=2000).show() #col 객체

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



# sql을 사용하기 위해서 view 등록 

In [17]:
df.createOrReplaceTempView('movies')  #view name, table name

In [18]:
#view 를 select projection하는 문장

query = '''
select name from movies
'''

spark.sql(query).show()


+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [27]:
#2010 년 이후 개봉한 영화 조회

query = """
select name, company
from movies
where year>=2010
"""

In [28]:
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|겨울왕국| 디즈니|
+--------+-------+



In [30]:
#회사가 마블인 영화 목록

query = """
select name, company
from movies
where company = '마블'
"""

In [32]:
#~맨
query = """
select name, company
from movies
where name like '%맨'
"""



In [33]:
# ~이~
query = """
select name, company
from movies
where name like '%이%'
"""


In [35]:
#개봉월이 4~8월 사이
uery = """
select name, company
from movies
where month between 4 and 8
"""


In [25]:
# and, or 
# ~맨, 개봉연도가 2010년 이하
where name like '%맨' and
year <= 2010

SyntaxError: invalid syntax (1710475871.py, line 3)

In [26]:
#회사가 마블, dc 인 영화
where company = '마블' or company = 'DC'
where company in ('마블','DC')

SyntaxError: invalid syntax (4018393345.py, line 2)

In [ ]:
# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화
where company like '마%' or company like '%니'

In [ ]:
# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화 중 2010년 이후로 개봉한 영화
where (company like '마%' or company like '%니') 
and year >= 2010

ASC : ascending 오름차순, default
DESC : descending 내림차순

In [ ]:
# 개봉 연도 오름차순으로 확인
from movies
order by year ASC

In [ ]:
#count(*)m count(name)
#mean, sum 
select count(*)

In [41]:
attendances = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [42]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [43]:
att_df = spark.createDataFrame( data=attendances, schema = att_schema )
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [44]:
att_df.show()

24/12/10 14:34:14 ERROR Executor: Exception in task 0.0 in stage 10.0 (TID 14)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/opt/spark/python/pyspark/sql/session.py", line 682, in prepare
    verify_func(obj)
  File "/opt/spark/python/pyspark/sql/types.py", line 1409, in verify
    verify_value(obj)
  File "/opt/spark/python/pyspark/sql/types.py", line 1386, in verify_struct
    raise ValueError(
ValueError: Length of object (2) does not match with length of fields (3)

	at org.

Py4JJavaError: An error occurred while calling o181.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 14) (ip-172-31-1-176.ap-northeast-3.compute.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/opt/spark/python/pyspark/sql/session.py", line 682, in prepare
    verify_func(obj)
  File "/opt/spark/python/pyspark/sql/types.py", line 1409

In [45]:
att_df.createOrReplaceTempView('att')

In [ ]:
att_df.select('*').show()

In [50]:
query = '''
select movies.id, movies.name, movies.company, att.att, att.theater_country
from movies JOIN att ON  movies.id = att.id
'''

In [51]:
moviews_views= spark.sql(query)

In [52]:
moviews_views.createOrReplaceTempView('moviews_views')

In [53]:
query = '''
select *from movies_view 
'''

In [54]:
spark.sql(query).show()

AnalysisException: Table or view not found: movies_view; line 2 pos 13;
'Project [*]
+- 'UnresolvedRelation [movies_view], [], false


In [ ]:
query = '''
select *
from movies_view 
where company =

'''

In [55]:
spark.stop()